In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle
from openpyxl import Workbook

# === CONFIG ===
start_date = datetime(2024, 1, 1, 0, 0, 0)
end_date = datetime(2024, 12, 31, 23, 0, 0)
time_range = pd.date_range(start=start_date, end=end_date, freq="H")

stations = ["Station_A", "Station_B", "Station_C", "Station_D", "Station_E"]
weather_conditions = ["Sunny", "Cloudy", "Rainy", "Windy", "Foggy"]

# === DEFINE TRAFFIC LEVEL ===
def traffic_level(value):
    if value <= 35:
        return "Low"
    elif value <= 75:
        return "Moderate"
    else:
        return "High"

# === GENERATE DATA ===
data = []
for timestamp in time_range:
    for station in stations:
        hour = timestamp.hour
        temp = np.random.normal(30, 5)
        humidity = np.random.randint(40, 90)
        weather = np.random.choice(weather_conditions, p=[0.4, 0.3, 0.15, 0.1, 0.05])
        traffic_index = np.clip(np.random.normal(60, 20), 10, 100)
        traffic_lvl = traffic_level(traffic_index)
        base_demand = np.random.randint(5, 25)
        if 7 <= hour <= 9 or 17 <= hour <= 20:
            base_demand *= 1.5
        elif 0 <= hour <= 5:
            base_demand *= 0.6
        if weather == "Rainy":
            base_demand *= 0.7
        elif weather == "Sunny":
            base_demand *= 1.1
        base_demand *= (traffic_index / 100)
        demand_units = max(0, np.round(base_demand + np.random.normal(0, 2), 1))
        data.append([
            timestamp,
            station,
            weather,
            round(temp, 1),
            humidity,
            round(traffic_index, 1),
            traffic_lvl,
            demand_units
        ])

df = pd.DataFrame(data, columns=[
    "Timestamp", "Station_ID", "Weather", "Temperature_C",
    "Humidity_%", "Traffic_Index", "Traffic_Level", "Charging_Demand_Units"
])

# === SAVE TO EXCEL WITH PROPER DATE FORMATTING ===
wb = Workbook()
ws = wb.active
ws.title = "EV_Charging_Demand"

# Write headers + data
for r in dataframe_to_rows(df, index=False, header=True):
    ws.append(r)

# Apply date format
date_style = NamedStyle(name="datetime_style", number_format="yyyy-mm-dd hh:mm:ss")
for cell in ws["A"][1:]:  # skip header
    cell.style = date_style

# Auto adjust column widths
for column_cells in ws.columns:
    length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in column_cells)
    ws.column_dimensions[column_cells[0].column_letter].width = length + 2

# Save
filename = "EV_Charging_Demand_BaseData_Final.xlsx"
wb.save(filename)
print(f" Excel dataset created successfully: {filename}")


In [ ]:
# === IMPORTS ===
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import os

# === LOAD BASE DATASET ===
base_file = "EV_Charging_Demand_BaseData_Final.xlsx"
if not os.path.exists(base_file):
    raise FileNotFoundError(f"{base_file} not found. Please ensure the dataset is generated first.")

df = pd.read_excel(base_file)

# === PREPARE DATA ===
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df = df.sort_values(["Station_ID", "Timestamp"])

# Prophet requires columns: ds (datetime) and y (value)
forecasts = []
forecast_plots_dir = "Forecast_Plots"
os.makedirs(forecast_plots_dir, exist_ok=True)

# === FORECAST PER STATION ===
for station in df["Station_ID"].unique():
    station_df = df[df["Station_ID"] == station].copy()
    
    prophet_df = station_df[["Timestamp", "Charging_Demand_Units"]].rename(
        columns={"Timestamp": "ds", "Charging_Demand_Units": "y"}
    )
    
    # Create and fit model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        changepoint_prior_scale=0.5
    )
    model.fit(prophet_df)
    
    # Future dataframe for next 30 days (hourly)
    future = model.make_future_dataframe(periods=24 * 30, freq="H")
    forecast = model.predict(future)
    
    # Add Station ID and merge back other relevant info
    forecast["Station_ID"] = station
    forecast["Traffic_Level"] = station_df["Traffic_Level"].mode()[0]  # most frequent level
    
    forecasts.append(forecast)
    
    # Plot and save
    fig = model.plot(forecast, xlabel="Date", ylabel="Charging Demand (kWh)")
    plt.title(f"{station} - EV Charging Demand Forecast", fontsize=14)
    plt.savefig(f"{forecast_plots_dir}/{station}_Forecast.png", bbox_inches="tight")
    plt.close(fig)

# === COMBINE ALL FORECASTS ===
forecast_all = pd.concat(forecasts, ignore_index=True)

# === SAVE TO EXCEL ===
output_file = "EV_Charging_Demand_Forecast_AllStations.xlsx"
forecast_all.to_excel(output_file, index=False)

print(f" Forecast completed for all stations!")
print(f" Results saved in: {output_file}")
print(f" Forecast plots saved in: {forecast_plots_dir}/")
print(forecast_all.head(10))


In [ ]:
"""
EV Charging Demand - Auto Report Generator
Generates project visuals, a PowerPoint, and a PDF report using only the relevant charts:
  - Actual vs Forecast (line) per station
  - Hourly Demand Heatmap (station vs hour)
  - Weather impact (bar)
  - Traffic Level impact (bubble)
  - KPI summary slide/page
"""

# Install missing Jupyter package dependency for pptx (runs in the notebook)
%pip install python-pptx --quiet
%pip install reportlab --quiet

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from reportlab.platypus import SimpleDocTemplate, Image, Spacer, Paragraph
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.styles import getSampleStyleSheet

# ---------------------------
# Config / Input filenames
# ---------------------------
BASE_FILE = "EV_Charging_Demand_BaseData_Final.xlsx"
FORECAST_FILE = "EV_Charging_Demand_Forecast_AllStations.xlsx"

OUTPUT_CHART_DIR = "EV_Visualization_Charts"
PPTX_FILE = "EV_Charging_Demand_Report.pptx"
PDF_FILE = "EV_Charging_Demand_Report.pdf"

# Ensure output folder
os.makedirs(OUTPUT_CHART_DIR, exist_ok=True)

# ---------------------------
# Load data
# ---------------------------
if not os.path.exists(BASE_FILE) or not os.path.exists(FORECAST_FILE):
    raise FileNotFoundError("Please ensure base & forecast Excel files exist: "
                            f"{BASE_FILE}, {FORECAST_FILE}")

base_df = pd.read_excel(BASE_FILE)
forecast_df = pd.read_excel(FORECAST_FILE)

# Normalize datetime columns
base_df["Timestamp"] = pd.to_datetime(base_df["Timestamp"])
# Forecast may have 'ds' or 'Timestamp'
if "ds" in forecast_df.columns:
    forecast_df = forecast_df.rename(columns={"ds": "Timestamp", "yhat": "Forecast_Demand_Units"})
else:
    forecast_df = forecast_df.rename(columns={"yhat": "Forecast_Demand_Units"})
forecast_df["Timestamp"] = pd.to_datetime(forecast_df["Timestamp"])

# Merge forecasts into base for easy plotting (outer join to keep forecast-only rows)
merged = pd.merge(
    base_df,
    forecast_df[["Timestamp", "Station_ID", "Forecast_Demand_Units", "Traffic_Level"]],
    on=["Timestamp", "Station_ID", "Traffic_Level"],
    how="outer",
    sort=True
)

# Feature engineering
merged["Hour"] = merged["Timestamp"].dt.hour
merged["Date"] = merged["Timestamp"].dt.date
merged["Month"] = merged["Timestamp"].dt.strftime("%B")
merged["Data_Type"] = np.where(merged["Forecast_Demand_Units"].isna(), "Actual", "Forecast")

# ---------------------------
# Utility: Save figures with consistent size
# ---------------------------
def save_fig(fig, filename, dpi=150):
    fig.savefig(filename, bbox_inches="tight", dpi=dpi)
    plt.close(fig)

# ---------------------------
# 1) Line: Actual vs Forecast per station
# ---------------------------
station_list = merged["Station_ID"].dropna().unique().tolist()
line_chart_files = []

for station in station_list:
    tmp = merged[merged["Station_ID"] == station].sort_values("Timestamp")
    # If both actual and forecast exist for same timestamps, plot both
    fig, ax = plt.subplots(figsize=(12, 4))
    # plot actual
    if tmp["Charging_Demand_Units"].notna().any():
        ax.plot(tmp["Timestamp"], tmp["Charging_Demand_Units"], label="Actual Demand", linewidth=1.5)
    # plot forecast
    if tmp["Forecast_Demand_Units"].notna().any():
        ax.plot(tmp["Timestamp"], tmp["Forecast_Demand_Units"], label="Forecast Demand", linestyle="--", linewidth=1.2)
    ax.set_title(f"{station} — Actual vs Forecast Charging Demand", fontsize=12)
    ax.set_xlabel("Timestamp")
    ax.set_ylabel("Demand (Units)")
    ax.legend()
    ax.grid(alpha=0.2)
    filename = os.path.join(OUTPUT_CHART_DIR, f"{station}_Actual_vs_Forecast.png")
    save_fig(fig, filename)
    line_chart_files.append(filename)

# ---------------------------
# 2) Heatmap: Hourly demand intensity (Station vs Hour)
# ---------------------------
heatmap_df = merged.groupby(["Station_ID", "Hour"], as_index=False)["Charging_Demand_Units"].mean().fillna(0)
pivot = heatmap_df.pivot(index="Station_ID", columns="Hour", values="Charging_Demand_Units").fillna(0)

fig, ax = plt.subplots(figsize=(14, max(4, 0.5 * len(pivot))))
sns.heatmap(pivot, ax=ax, cmap="YlGnBu")
ax.set_title("Hourly Charging Demand Intensity (Avg) — Station vs Hour")
ax.set_xlabel("Hour of Day")
ax.set_ylabel("Station_ID")
heatmap_file = os.path.join(OUTPUT_CHART_DIR, "Hourly_Demand_Heatmap.png")
save_fig(fig, heatmap_file)

# ---------------------------
# 3) Weather impact bar chart
# ---------------------------
weather_df = merged.groupby("Weather", as_index=False)["Charging_Demand_Units"].mean().dropna()
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(x="Weather", y="Charging_Demand_Units", data=weather_df, ax=ax)
ax.set_title("Average Charging Demand by Weather Type")
ax.set_xlabel("Weather")
ax.set_ylabel("Avg Demand (Units)")
weather_file = os.path.join(OUTPUT_CHART_DIR, "Weather_Impact.png")
save_fig(fig, weather_file)

# ---------------------------
# 4) Traffic level bubble chart
# ---------------------------
traffic_df = merged.groupby("Traffic_Level", as_index=False).agg({
    "Charging_Demand_Units": "mean",
    "Traffic_Index": "mean"
}).dropna()

fig, ax = plt.subplots(figsize=(8, 5))
colors = {"Low": "#2ca02c", "Moderate": "#ff7f0e", "High": "#d62728"}
x = traffic_df["Traffic_Index"]
y = traffic_df["Charging_Demand_Units"]
sizes = (y - y.min() + 1) * 80  # bubble sizing

ax.scatter(x, y, s=sizes, alpha=0.65, edgecolors="k",
           c=[colors.get(t, "#7f7f7f") for t in traffic_df["Traffic_Level"]])
for i, lvl in enumerate(traffic_df["Traffic_Level"]):
    ax.text(x.iloc[i] + 0.5, y.iloc[i], lvl, fontsize=10)
ax.set_title("Traffic Level vs Average Charging Demand")
ax.set_xlabel("Traffic Index (avg)")
ax.set_ylabel("Avg Charging Demand (Units)")
traffic_file = os.path.join(OUTPUT_CHART_DIR, "Traffic_Impact.png")
save_fig(fig, traffic_file)

# ---------------------------
# KPI summary (for slide and pdf text)
# ---------------------------
avg_demand = merged["Charging_Demand_Units"].mean()
peak_demand_row = merged.loc[merged["Charging_Demand_Units"].idxmax()] if merged["Charging_Demand_Units"].notna().any() else None
peak_demand = peak_demand_row["Charging_Demand_Units"] if peak_demand_row is not None else np.nan
peak_hour = int(peak_demand_row["Hour"]) if peak_demand_row is not None else None
dominant_weather = merged["Weather"].mode().iloc[0] if not merged["Weather"].dropna().empty else None
dominant_traffic = merged["Traffic_Level"].mode().iloc[0] if not merged["Traffic_Level"].dropna().empty else None

# Save a tiny KPI image (simple text image) for slide
kpi_text = [
    f"Average Charging Demand: {avg_demand:.2f} units",
    f"Peak Charging Demand: {peak_demand:.2f} units at Hour: {peak_hour}:00" if not np.isnan(peak_demand) else "Peak Charging Demand: N/A",
    f"Most common weather: {dominant_weather}",
    f"Most frequent traffic level: {dominant_traffic}"
]
# Create KPI image
fig, ax = plt.subplots(figsize=(8, 3))
ax.axis("off")
txt = "\n".join(kpi_text)
ax.text(0.01, 0.95, "EV Charging Demand — KPI Summary", fontsize=14, weight="bold", va="top")
ax.text(0.01, 0.6, txt, fontsize=11, va="top")
kpi_file = os.path.join(OUTPUT_CHART_DIR, "KPI_Summary.png")
save_fig(fig, kpi_file)

# ---------------------------
# Build PowerPoint
# ---------------------------
prs = Presentation()
# set slide title style helper
def add_title_slide(prs, title, subtitle=None):
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # blank-like layout
    left = Inches(0.5)
    top = Inches(0.3)
    width = Inches(9)
    tx_box = slide.shapes.add_textbox(left, top, width, Inches(1.2))
    tf = tx_box.text_frame
    p = tf.paragraphs[0]
    p.text = title
    p.font.size = Pt(28)
    p.font.bold = True
    if subtitle:
        p2 = tf.add_paragraph()
        p2.text = subtitle
        p2.font.size = Pt(14)
    return slide

# Title slide
add_title_slide(prs, "EV Charging Demand Forecast Report", "Based on Time, Weather & Traffic")

# KPI slide
slide = prs.slides.add_slide(prs.slide_layouts[5])
left = Inches(0.5); top = Inches(0.6)
slide.shapes.add_textbox(left, top, Inches(9), Inches(0.6)).text_frame.text = "KPI Summary"
slide.shapes.add_picture(kpi_file, Inches(0.5), Inches(1.2), width=Inches(9))

# Heatmap slide
slide = prs.slides.add_slide(prs.slide_layouts[5])
slide.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(9), Inches(0.4)).text_frame.text = "Hourly Demand Heatmap"
slide.shapes.add_picture(heatmap_file, Inches(0.5), Inches(0.9), width=Inches(9))

# Weather & Traffic slides
slide = prs.slides.add_slide(prs.slide_layouts[5])
slide.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(9), Inches(0.4)).text_frame.text = "Weather Impact"
slide.shapes.add_picture(weather_file, Inches(0.5), Inches(0.9), width=Inches(5))
slide.shapes.add_picture(traffic_file, Inches(5.7), Inches(0.9), width=Inches(3.8))

# Per-station slides (Actual vs Forecast)
for img in line_chart_files:
    station_name = os.path.basename(img).split("_Actual_vs_Forecast")[0]
    slide = prs.slides.add_slide(prs.slide_layouts[5])
    slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(0.4)).text_frame.text = f"{station_name} — Actual vs Forecast"
    slide.shapes.add_picture(img, Inches(0.5), Inches(0.8), width=Inches(9))

# Save presentation
prs.save(PPTX_FILE)
print(f"PowerPoint saved: {PPTX_FILE}")

# ---------------------------
# Build PDF using ReportLab
# ---------------------------
doc = SimpleDocTemplate(PDF_FILE, pagesize=landscape(A4))
styles = getSampleStyleSheet()
story = []

# Helper to add image full-width with small spacer
def add_image_page(img_path, title=None):
    if title:
        story.append(Paragraph(title, styles['Heading2']))
    im = Image(img_path, width=landscape(A4)[0] - 40, height=None)
    story.append(im)
    story.append(Spacer(1, 12))

# Title page
story.append(Paragraph("EV Charging Demand Forecast Report", styles['Title']))
story.append(Spacer(1, 12))
for line in kpi_text:
    story.append(Paragraph(line, styles['Normal']))
story.append(Spacer(1, 24))

# Add heatmap + weather & traffic + station pages
add_image_page(heatmap_file, "Hourly Demand Heatmap")
add_image_page(weather_file, "Weather Impact on Demand")
add_image_page(traffic_file, "Traffic Level Impact on Demand")

# Add station pages
for img in line_chart_files:
    station_name = os.path.basename(img).split("_Actual_vs_Forecast")[0]
    add_image_page(img, f"{station_name} — Actual vs Forecast")

doc.build(story)
print(f"PDF saved: {PDF_FILE}")

print("\n All report files generated:")
print(f" - Charts folder: {OUTPUT_CHART_DIR}")
print(f" - PPTX: {PPTX_FILE}")
print(f" - PDF: {PDF_FILE}")
